# Преобразование Фурье

Рассмотреть сигнал  синусоидальной формы с частотой 120 Гц, Амплитудой 1 В. Отсчеты сигнала реализовать через каждую миллисекунду. Длина выборки 1000 отсчетов.  Получить Фурье-образ сигнала в зависимости от частоты отсчетов. Определить амплитуду Фурье-гармоники на частоте 120 Гц. Сравнить в амплитудой, которая была бы получена аналитическим путем.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

plt.rcParams['font.serif'] = ['Hack']

In [ ]:
freq = 120 # Гц
Ampl = 1.0 # В
dt = 0.001 # период дисректизации, с
N = 1000 # длина выборки 
nu = 1 / dt # частота дискретизации


t = np.arange(N) * dt
signal = Ampl * np.sin(2 * np.pi * freq * t)

fft_signal = np.fft.fft(signal)